# Text Mining 
## Hand-in Assignment 1

Student: Elin Benja Dijkstra

Studentnumber: S2696096

Deadline: 20-10-2019

1. The	tutorial	classifies	between only	four	categories	of	the	20newsgroups	data	set.	Change	
your	script	so	that	it	addresses all	20	categories.
DONE

2. Compare	three	classifiers	on	this	multi-class	classification	task,	including	at	least	Naïve	Bayes. DONE
3. Compare	three	type	of	features for	your	classifiers:	counts,	tf,	and	tf-idf.
4. Look	up	the	documentation	of	the	CountVectorizer function	and	experiment	with	different	
values	for	the	following	parameters:	
a. lowercase
b. stop_words
c. analyzer	(in	combination	with	ngram_range)
d. max_features
5. Write	one	script	for	running these	experiments and	printing	the	results.

## Import needed packages

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Loading and examining the dataset

To complete the first assignment 1.1) we simply remove the categories = categories so we no longer get a subset but all 20 categories in the dataset.

In [3]:
# define our training and testing set
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

## Text Tokenization

In [4]:
# initialize our count vector with different features as described in assignment 1.4)
def experiment_count_vect(lower_case_bool, stop_words, analyzer, max_features):
    return CountVectorizer(lowercase = lower_case_bool, stop_words = stop_words, 
                                 analyzer = analyzer, max_features = max_features)

In [5]:
# tokenize our data depending on the different features for our classifiers defined in 1.3)
def tokenize(count_vect, feature_type):
    X_train_counts = count_vect.fit_transform(twenty_train.data)
    if feature_type == "tf":
        tfidf_transformer = TfidfTransformer(use_idf=False)
        X_train_tf = tfidf_transformer.fit_transform(X_train_counts)
        return X_train_tf
    elif feature_type == "tfidf":
        tfidf_transformer = TfidfTransformer()
        X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
        return X_train_tfidf
    else: 
        return X_train_counts

## Training a classifier

#### Choose one of three classifiers: Naive Bayes, Stochastic Gradient Descent of LogisticRegression/Support vector classifier

In [19]:
def train_classifier(type_classifier):
    if type_classifier == "Naive Bayes":
#         clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', MultinomialNB()),
        ])
    elif type_classifier == "SGD":
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])
    elif type_classifier == "Logistic Regression":
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', LogisticRegression(penalty='l2', multi_class = "ovr")),
        ])
        
    elif type_classifier == "SVM":
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SVC(decision_function_shape = "ovr")),
        ])
    return text_clf

## Evaluation on the test set

In [20]:
def evaluate(text_clf):
    docs_test = twenty_test.data
    predicted = text_clf.predict(docs_test)
    np.mean(predicted == twenty_test.target) 
    
    print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))
    print("Confusion matrix: ")
    cm = metrics.confusion_matrix(twenty_test.target, predicted)
#     print(cm)

In [21]:
def start_experiment(lower_case, stop_words, analyzer, max_features, feature_type, classifiers):
    count_vect = experiment_count_vect(lower_case, stop_words, analyzer, max_features)
    tokenize(count_vect, feature_type)
    for classifier in classifiers:
        print(color.BOLD + '\n Evaluation for' + str(classifier) + '\n' + color.END)
        text_clf = train_classifier(classifier)
        fit = text_clf.fit(twenty_train.data, twenty_train.target)  
        evaluate(fit)

In [22]:
for lower_case in lower_case_set:
    for stop_words in stop_words_set:
        for analyzer in analyzer_set:
            for max_features in max_features_set:
                for feature_type in feature_type_set:
                    print("Parameters used:" + " lowercase:"+ str(lower_case)+ "\nstop_words: "+ str(stop_words)+
                          "\nanalyzer: "+ str(analyzer)+ "\nmax_features: "+ str(max_features)+ "\nfeature_type: " 
                          + str(feature_type))
                    start_experiment(lower_case, stop_words, analyzer, max_features, feature_type, classifiers)

Parameters used: lowercase:False
stop_words: None
analyzer: char
max_features: 1000
feature_type: tfidf

 Evaluation forLogistic Regression



C:\Users\elind\Anaconda2\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


                          precision    recall  f1-score   support

             alt.atheism       0.80      0.74      0.77       319
           comp.graphics       0.69      0.78      0.74       389
 comp.os.ms-windows.misc       0.76      0.75      0.75       394
comp.sys.ibm.pc.hardware       0.73      0.72      0.72       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.76      0.90      0.83       390
               rec.autos       0.91      0.89      0.90       396
         rec.motorcycles       0.94      0.95      0.94       398
      rec.sport.baseball       0.87      0.93      0.90       397
        rec.sport.hockey       0.94      0.96      0.95       399
               sci.crypt       0.93      0.89      0.91       396
         sci.electronics       0.76      0.78      0.77       393
                 sci.med       0.89      0.84      0.86       396
         

C:\Users\elind\Anaconda2\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


                          precision    recall  f1-score   support

             alt.atheism       0.80      0.74      0.77       319
           comp.graphics       0.69      0.78      0.74       389
 comp.os.ms-windows.misc       0.76      0.75      0.75       394
comp.sys.ibm.pc.hardware       0.73      0.72      0.72       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.76      0.90      0.83       390
               rec.autos       0.91      0.89      0.90       396
         rec.motorcycles       0.94      0.95      0.94       398
      rec.sport.baseball       0.87      0.93      0.90       397
        rec.sport.hockey       0.94      0.96      0.95       399
               sci.crypt       0.93      0.89      0.91       396
         sci.electronics       0.76      0.78      0.77       393
                 sci.med       0.89      0.84      0.86       396
         

C:\Users\elind\Anaconda2\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


                          precision    recall  f1-score   support

             alt.atheism       0.80      0.74      0.77       319
           comp.graphics       0.69      0.78      0.74       389
 comp.os.ms-windows.misc       0.76      0.75      0.75       394
comp.sys.ibm.pc.hardware       0.73      0.72      0.72       392
   comp.sys.mac.hardware       0.81      0.83      0.82       385
          comp.windows.x       0.83      0.74      0.78       395
            misc.forsale       0.76      0.90      0.83       390
               rec.autos       0.91      0.89      0.90       396
         rec.motorcycles       0.94      0.95      0.94       398
      rec.sport.baseball       0.87      0.93      0.90       397
        rec.sport.hockey       0.94      0.96      0.95       399
               sci.crypt       0.93      0.89      0.91       396
         sci.electronics       0.76      0.78      0.77       393
                 sci.med       0.89      0.84      0.86       396
         

C:\Users\elind\Anaconda2\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


KeyboardInterrupt: 

In [ ]:
parameters = {
    'lower_case': (True, False)
    'stop_words':  ('english', None) 
    'analyzer': ['word', 'char', 'char_wb']
    'max_features': (100, 1000, 10000)
    'feature_type': ['counts', 'tf', 'tfidf']
    'classifiers': ['Naive Bayes', 'SGD', 'Logistic Regression', 'SVM']      
}

## Parameter tuning using grid search

In [ ]:
gs_clf.best_score_
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
pd.DataFrame(gs_clf.cv_results_)